Testing the celeb recognition package

https://pypi.org/project/celeb-detector/


https://www.kaggle.com/code/vinayakshanawad/celebrity-face-recognition-vggface-model/notebook 

In [4]:
import os
import random
import shutil
import face_recognition
import numpy as np

# Define paths for source, training, and test directories
source_dir = "celebrities"
train_dir = "celebrities_train"
test_dir = "celebrities_test"

# Set split ratio
test_ratio = 0.2

# Step 1: Split celebrity folders into training and test sets
def split_data():
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    
    # Iterate through each celebrity folder
    for celeb_folder in os.listdir(source_dir):
        celeb_path = os.path.join(source_dir, celeb_folder)
        
        if os.path.isdir(celeb_path):
            # List all image files
            images = [img for img in os.listdir(celeb_path) if img.endswith(('.jpg', '.jpeg'))]
            random.shuffle(images)
            
            # Split images
            split_index = int(len(images) * (1 - test_ratio))
            train_images = images[:split_index]
            test_images = images[split_index:]
            
            # Create directories in train/test folders
            train_celeb_folder = os.path.join(train_dir, celeb_folder)
            test_celeb_folder = os.path.join(test_dir, celeb_folder)
            os.makedirs(train_celeb_folder, exist_ok=True)
            os.makedirs(test_celeb_folder, exist_ok=True)
            
            # Move images to train and test folders
            for img in train_images:
                shutil.copy(os.path.join(celeb_path, img), train_celeb_folder)
            for img in test_images:
                shutil.copy(os.path.join(celeb_path, img), test_celeb_folder)

# Step 2: Encode faces in the training set
def encode_training_faces():
    encodings = {}
    
    for celeb_folder in os.listdir(train_dir):
        celeb_path = os.path.join(train_dir, celeb_folder)
        if os.path.isdir(celeb_path):
            celeb_encodings = []
            
            # Process each image
            for img_name in os.listdir(celeb_path):
                img_path = os.path.join(celeb_path, img_name)
                img = face_recognition.load_image_file(img_path)
                
                # Get encodings (assuming one face per image)
                encoding = face_recognition.face_encodings(img)
                if encoding:
                    celeb_encodings.append(encoding[0])
            
            # Save encodings for each celebrity
            if celeb_encodings:
                encodings[celeb_folder] = np.mean(celeb_encodings, axis=0)
    
    return encodings


In [19]:

# Step 3: Identify a random celebrity photo from the test set
def identify_random_test_image(encodings):
    celeb_folder = random.choice(os.listdir(test_dir))
    celeb_path = os.path.join(test_dir, celeb_folder)
    test_image = random.choice(os.listdir(celeb_path))
    test_image_path = os.path.join(celeb_path, test_image)
    
    # Load and encode test image
    test_img = face_recognition.load_image_file(test_image_path)
    test_encoding = face_recognition.face_encodings(test_img)
    
    if test_encoding:
        test_encoding = test_encoding[0]
        
        # Compare with known encodings
        results = face_recognition.compare_faces(
            list(encodings.values()), test_encoding, tolerance=0.6
        )
        
        # Find match
        if True in results:
            match_index = results.index(True)
            matched_celebrity = list(encodings.keys())[match_index]
            print(f"Identified as: {matched_celebrity} from test image: {test_image_path}")
        else:
            print(f"No match found for {test_image_path}.")
    else:
        print(f"No face detected in test image {test_image_path}")

# Run steps
split_data()
celebrity_encodings = encode_training_faces()


In [52]:

identify_random_test_image(celebrity_encodings)


No face detected in test image celebrities_test/Pedro-Pascal/58.jpg
